In [1]:
cd /specific/a/home/cc/students/math/raphaelf/thesis

/specific/a/home/cc/students/math/raphaelf/thesis


In [2]:
from data_helper import files_df, split_train_test
from model import PDFDataSet, MalConv
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn

In [3]:
df = files_df()
df_train, df_test = split_train_test(df, 0.1)
df_test, df_valid = split_train_test(df_test, 0.2)

In [4]:
set(df_train.index).intersection(set(df_test.index))

set()

In [5]:
dataloader = DataLoader(PDFDataSet(df_test), batch_size=64, shuffle=False)
device = "cuda:3"

In [6]:
malconv = torch.load('Chkp/5f0bfc8b-458d-45f7-b72c-bdafeb7c5703')
malconv.eval()
malconv = malconv.to(device)

In [7]:
val_pred = []
val_label = []
for _,val_batch_data in enumerate(dataloader):
    cur_batch_size = val_batch_data[0].size(0)

    exe_input = val_batch_data[0].to(device)
    exe_input = Variable(exe_input.long(),requires_grad=False)

    label = val_batch_data[1].to(device)
    label = Variable(label.float(),requires_grad=False)

    pred = malconv(exe_input)
    val_pred.extend(pred.cpu().data)
    val_label.extend(label)

In [9]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [41]:
cm = confusion_matrix(val_label, np.array(val_pred) > 0.1)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm

array([[0.99056503, 0.00943497],
       [0.02631579, 0.97368421]])

In [14]:
import pandas as pd

In [31]:
threshold = .5

In [15]:
df_results = df_test.copy()

In [18]:
df_results['pred'] = pd.Series(np.array(val_pred), index=df_results.index)

In [37]:
fn = df_results[(df_results['pred'] <= threshold) & (df_results['verdict'])]
print("\n".join(fn['hash'].values[:50]))

d8c7998c1db522989d8e4ee37a6e136ba4f5d3a57d322a7351067eeabebd38f0
1e4fa4d38962eaab913d71d1aec3b339464223feb42eee2064a580dd25b54c74
893e9a521f1b74f9f0b663f506179fdb0e2d8636218c8c5b4423f5b804fb5aba
381acaee65197d2336618a5866882e1afc8ca77b2c3515a2de9b60572bf7e7a0
6d0bf202d0e7e15a146cb4d2bdff528f59f275e239d4928428c09cd0ac60bd43
9057e08f37250143fcbd4188be782314badd874d3d5b11ae6596703e47831c52
973b8ccea7bb1376f7d377eddf3a6728433b2357946d82b62f69d1a6a01a8e69
1a882542a5154a41f2e29556939cf57e1810fc295737e036bb652cf596768b90
523e2752ef8af9b87590f0dd7816e76f82c94ec2072e9d21054e34af513f74a2
0fe423f447afa2a4f02a2c8f840d52896851a9c79216acde2e48c1619f114bd1
4eec94849b25f46c677f0c8a9386ba30f645c67586dd11cee57cc9f6e21cdce3
f74dc9c87e7ca55310a59906d9797746f70cf297e2a41e57d5eceaf618046e27
ba2b524b889bbdd1df44c7ea77542a93a6c650c9806f30e2153f9180a67b02e7
1aba2082bf8f5e6f5fe2ba4deaa2875b92b193999ddccce449cd0d3c61f8cf32
b61cfd47c51b4a4cbdae7f7da0c7765cc27bd6d875fabd47b31aaeb1efda3730
dd6ac22cf41bb73d5162d357f

In [40]:
tp = df_results[(df_results['pred'] > threshold) & (df_results['verdict'])]
print("\n".join(tp['hash'].values[:50]))

8ee260b86091e679a1335bf4f0000e7d498f584be5c80a2b18dc7606cb323d42
443effa6a5dbdbee81048798311f8e9442b27b1b22ba20d9d91a2ad659950555
3eaaa89ae7424e8be7f357c49e3afdc57c439788269c159919f68c61a5ce20b9
56457c0254fd936116843e16d5bed6acdd2918dedd14de305f69a0e8e9a8d788
682889440e590240c476ebe8ead29dd2af80f5539408eef32caebf0393ff95c2
369e03253c072b0c406cb3d817441a144f49d255675e18611a488ddf1f3d9399
cbf2c68dfe9c0f2850163a300ee7235c798ddbf11a4a7bccf89567abdbc1993d
193327cb2e14550a8ac655a336e6bf46732f6a72b9facc274455f576e1c28f80
0bb567bbcb8cdfa5074301580f43188b4780a4fbab3f709fe37259293cd94a34
cdcbd95b59e0db613a88901eea3c93893f2c55f03786d33ed583d1164f1fdf55
933aa0aa501ab1106ae25231eaeb7c5138444f3b8eeadc3f16dd06698c2a0f99
c86e7c75187d5059c2d8c3519e2c019298835bc44e828476ecc4f6b372470685
8d6423bfc7e8e16f96ba7cf623ba3e1c0a13da559e7fa25f3fef7e61c2492ecc
25dca60de62f8bffefb912a7947653d62b246e2243f95b69ba3c7eaec61c7e03
0670c7ce7fede0824bcb864ce5503d5903066620c5b7e868695e771cd153876b
6ebdf0f0636b88ca7c478e44a